# LLM Providers

TimeCopilot is designed to be flexible and work with a variety of Large Language Model (LLM) providers. This notebook demonstrates how to configure TimeCopilot to work with different LLM providers, giving you the freedom to choose the model that best fits your needs, budget, and infrastructure requirements.

Whether you prefer cloud-based solutions like OpenAI, self-hosted models via Ollama, or other compatible providers, TimeCopilot can adapt to your setup. This flexibility allows you to:
- **Control costs** by choosing providers that fit your budget
- **Ensure privacy** by using self-hosted models
- **Optimize performance** by selecting models that match your use case
- **Leverage existing infrastructure** by using providers you already have configured

# Import Libraries


Let's start by importing the necessary libraries. We'll use `nest_asyncio` to run `TimeCopilot` in Jupyter notebooks, and `pandas` for data manipulation. The `TimeCopilot` class is the main interface we'll use for forecasting.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import pandas as pd
from timecopilot import TimeCopilot


# Loading Environment Variables

TimeCopilot uses [Pydantic AI](https://ai.pydantic.dev/) to interact with LLMs, which provides excellent support for [many common model providers](https://ai.pydantic.dev/models/overview/) and also supports any endpoint that follows the OpenAI API format.

## Why Environment Variables?

Most LLM providers require API keys or authentication tokens to access their services. For security reasons, these credentials should never be hardcoded in your scripts. Instead, they should be stored as environment variables, which are loaded at runtime.

## Provider-Specific Configuration

Configuration requirements vary depending on which provider you're using:
- **OpenAI**: Requires `OPENAI_API_KEY` environment variable
- **Ollama**: Requires `OLLAMA_BASE_URL` and optionally `OLLAMA_API_KEY`
- **Other providers**: Each has its own specific environment variables (see Pydantic's documentation for details)

Below are three common methods for setting environment variables. Choose the one that best fits your workflow.

### Method 1: Directly Set in Python

This method is useful for quick testing or when you need to set variables programmatically. **Note**: This is less secure for production use, as the key is visible in your code.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-new-secret-key"

### Method 2: Set in Your Shell

This method sets the environment variable in your terminal session. The variable will be available to all processes started from that terminal, but will be lost when you close the terminal.

**Important**: This needs to be done before starting your Python process (before running `jupyter notebook`, `python script.py`, etc.).

In [ ]:
# bash
export OPENAI_API_KEY="your-new-secret-key"

In [ ]:
# powershell
setx OPENAI_API_KEY "your-new-secret-key"

### Method 3: Store in a .env File (Recommended)

This is the most convenient and secure method for development. You create a `.env` file in your project directory (make sure to add it to `.gitignore` to avoid committing secrets), and load it using the `python-dotenv` package.

**Benefits**:
- Keeps credentials out of your code
- Easy to manage multiple environment variables
- Works consistently across different operating systems
- Can be easily excluded from version control

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

# Supported LLM Providers

TimeCopilot supports all LLM providers that are compatible with Pydantic AI. For a comprehensive list of officially supported providers, visit [Pydantic's model providers overview](https://ai.pydantic.dev/models/overview/).

Each provider has its own setup requirements and configuration options, which are documented in Pydantic's documentation. In this notebook, we'll demonstrate a few common providers:

1. **OpenAI** - Cloud-based, widely used, excellent performance
2. **Ollama** - Self-hosted, privacy-focused, runs models locally
3. **OpenAI-compatible endpoints** - Any service that follows the OpenAI API format

Let's explore each of these options with practical examples.

## OpenAI

OpenAI provides some of the most powerful and widely-used language models, including GPT-5.1, GPT-4, GPT-3.5, and GPT-4o. These models offer excellent performance for TimeCopilot's forecasting tasks.

### When to Use OpenAI
- You need high-quality model outputs
- You're comfortable with cloud-based solutions
- You have an OpenAI API account and budget
- You want reliable, production-ready models

### Set the Environment Variable

`OPENAI_API_KEY` is the required environment variable for OpenAI. You can get your API key from the [OpenAI Platform](https://platform.openai.com/api-keys).

Load it with your preferred method (the three methods are described above). Here's an example using the `.env` file approach:

In [ ]:
# load the .env file with python-dotenv
from dotenv import load_dotenv

load_dotenv()

### Initialize the Forecasting Agent

Now that your API key is configured, you can initialize TimeCopilot with an OpenAI model. The `llm` parameter accepts a string in the format `"provider:model-name"`. For OpenAI, you can use models like:
- `"openai:gpt-4o"` - Latest GPT-4 optimized model (recommended)
- `"openai:gpt-4"` - GPT-4 model
- `"openai:gpt-3.5-turbo"` - Faster and more cost-effective option

The `retries` parameter specifies how many times TimeCopilot should retry failed API calls, which helps with reliability.


In [4]:
tc = TimeCopilot(
    llm="openai:gpt-4o",
    retries=3
)

## Ollama

Ollama is a powerful tool for running large language models locally on your own machine. This gives you complete control over your data and eliminates API costs, making it ideal for:
- **Privacy-sensitive applications** - Your data never leaves your machine
- **Cost optimization** - No per-request API fees
- **Offline usage** - Works without internet connectivity
- **Custom models** - Run specialized or fine-tuned models

### When to Use Ollama
- You have sufficient local compute resources (GPU recommended)
- Privacy is a primary concern
- You want to avoid API costs
- You need to work with custom or specialized models

### Set the Environment Variables

Ollama requires the `OLLAMA_BASE_URL` environment variable, and optionally `OLLAMA_API_KEY` depending on your configuration:

- **Local installation**: If you're running Ollama locally, you typically don't need an API key. The default URL is `http://localhost:11434/v1`
- **Ollama Cloud**: If you're using Ollama's cloud service, you'll need both variables with `OLLAMA_BASE_URL` set to `https://ollama.com/v1`

To get started with Ollama locally:
1. Install Ollama from [ollama.ai](https://ollama.ai)
2. Pull a model: `ollama pull gpt-oss:20b` (or any other model)
3. Set the environment variables as shown below

Load your environment variables with your preferred method (the three methods are described above): 

In [ ]:
# load the .env file with python-dotenv
from dotenv import load_dotenv

load_dotenv()

### Initialize the Forecasting Agent

There are two ways to initialize TimeCopilot with Ollama. The first is simpler and uses a string format, while the second gives you more control over the configuration.

#### Approach 1: Simple String Format (Recommended)

This is the easiest way to use Ollama. Just specify the provider and model name in the format `"ollama:model-name"`. TimeCopilot will automatically use the `OLLAMA_BASE_URL` from your environment variables.

In [ ]:
tc = TimeCopilot(
    llm='ollama:gpt-oss:20b',
    retries=3
)

#### Approach 2: Direct Provider Initialization

If you need more control over the configuration (for example, to use a different base URL than what's in your environment variables), you can initialize the model and provider directly. This approach is also useful when you want to programmatically set the base URL or use multiple Ollama instances.

In [ ]:
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.ollama import OllamaProvider

llm = OpenAIChatModel(
    model_name="gpt-oss:20b",
    provider=OllamaProvider(base_url="http://localhost:11434/v1"),
)

tc = TimeCopilot(
    llm=llm,
    retries=3
)

## OpenAI-Compatible Endpoints

Many LLM providers and self-hosted solutions offer APIs that are compatible with OpenAI's API format. This includes:
- **LocalAI** - Self-hosted OpenAI-compatible API
- **vLLM** - High-performance inference server
- **Text Generation Inference (TGI)** - Hugging Face's inference server
- **Custom deployments** - Any service that follows OpenAI's API specification

### When to Use OpenAI-Compatible Endpoints
- You're running your own inference server
- You want to use models not available through standard providers
- You need fine-grained control over the API endpoint
- You're using a provider that supports OpenAI's API format but isn't directly supported by Pydantic

### How It Works

These providers use OpenAI's API format, so we can use Pydantic's `OpenAIChatModel` with a custom `OpenAIProvider` that points to your endpoint. This gives you the flexibility to use any OpenAI-compatible service with TimeCopilot.

### Example: Using a Local OpenAI-Compatible Endpoint

The following example shows how to configure TimeCopilot to use a local inference server running on `http://127.0.0.1:1234/v1`. Adjust the `LLM_API_BASE_URL`, `MODEL_NAME`, and `API_KEY` to match your setup.



In [ ]:
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openai import OpenAIProvider

LLM_API_BASE_URL = "http://127.0.0.1:1234/v1"
MODEL_NAME = "gpt-oss-20b"
API_KEY = "api-key"

model = OpenAIChatModel(
    MODEL_NAME,
    provider=OpenAIProvider(
        base_url=LLM_API_BASE_URL,
        api_key=API_KEY,
    ),
)

tc = TimeCopilot(
    llm=model,
    retries=3,
)

# Using TimeCopilot with Your Chosen Provider

Once you've configured your LLM provider and initialized TimeCopilot, you can use it exactly as you would with any other provider. The interface remains consistent regardless of which provider you choose, making it easy to switch between providers or test different models.

Let's demonstrate this with a practical example using the classic Air Passengers dataset. This will show how TimeCopilot:
1. Analyzes your time series data
2. Selects appropriate forecasting models
3. Generates forecasts
4. Answers natural language questions about the forecasts

First, let's load a sample time series dataset. The Air Passengers dataset contains monthly totals of international airline passengers from 1949 to 1960, which is a classic example for time series forecasting.

In [ ]:
df = pd.read_csv("https://timecopilot.s3.amazonaws.com/public/data/air_passengers.csv")
print(df.head())

       unique_id          ds    y
0  AirPassengers  1949-01-01  112
1  AirPassengers  1949-02-01  118
2  AirPassengers  1949-03-01  132
3  AirPassengers  1949-04-01  129
4  AirPassengers  1949-05-01  121


## Generate Forecasts

Now let's use TimeCopilot to generate forecasts. The `forecast` method will:
1. Analyze the time series characteristics
2. Select appropriate forecasting models based on the data
3. Train and evaluate multiple models
4. Return the best forecast along with detailed analysis

The `freq="MS"` parameter specifies that the data has monthly frequency (Month Start).


In [ ]:
result = tc.forecast(df=df, freq="MS")

1it [00:00,  7.48it/s]
1it [00:00, 25.92it/s]
1it [00:00, 251.31it/s]
0it [00:00, ?it/s]15:51:06 - cmdstanpy - INFO - Chain [1] start processing
15:51:06 - cmdstanpy - INFO - Chain [1] done processing
1it [00:03,  3.55s/it]
15:51:15 - cmdstanpy - INFO - Chain [1] start processing
15:51:15 - cmdstanpy - INFO - Chain [1] done processing
0it [00:00, ?it/s]15:51:18 - cmdstanpy - INFO - Chain [1] start processing
15:51:19 - cmdstanpy - INFO - Chain [1] done processing
1it [00:02,  2.97s/it]15:51:21 - cmdstanpy - INFO - Chain [1] start processing
15:51:21 - cmdstanpy - INFO - Chain [1] done processing
2it [00:05,  2.97s/it]15:51:24 - cmdstanpy - INFO - Chain [1] start processing
15:51:25 - cmdstanpy - INFO - Chain [1] done processing
3it [00:09,  3.03s/it]15:51:28 - cmdstanpy - INFO - Chain [1] start processing
15:51:28 - cmdstanpy - INFO - Chain [1] done processing
4it [00:12,  3.05s/it]15:51:31 - cmdstanpy - INFO - Chain [1] start processing
15:51:31 - cmdstanpy - INFO - Chain [1] done pro

## View the Analysis

TimeCopilot provides detailed analysis of your time series. The `tsfeatures_analysis` contains insights about:
- **Stability**: How stable the series is over time
- **Stationarity**: Whether the series has trends or is stationary
- **Seasonality**: The strength of seasonal patterns
- **Series characteristics**: Length, complexity, and other features

This analysis helps TimeCopilot select the most appropriate forecasting models for your data.

In [ ]:
print(result.output.tsfeatures_analysis)

The time series analysis reveals several key attributes which will guide our model selection:

1. **Stability (0.933)**: A higher value indicates a relatively stable time series, which is consistent with the seasonal pattern typical in a dataset like AirPassengers.
2. **Unit Root Tests**: The PP test (-6.57) suggests the series is stationary, whereas the KPSS test (2.74) indicates non-stationarity. This mixed signal is somewhat expected in real-world data and suggests models that handle both trends and seasonality might perform well.
3. **Trend and Seasonality**: The series includes a strong trend (0.997) and pronounced seasonal strength (0.982). This points to models that can effectively capture both components, such as Prophet, which explicitly models both trend and seasonality.
4. **Series Length (144)**: A reasonably long series enables the usage of sophisticated models that require more data to perform effectively.


## View the Forecasts

The `fcst_df` contains the actual forecast values. Each row represents a future time point with the predicted value from the selected model (in this case, Prophet was chosen based on the time series characteristics).

In [ ]:
print(result.fcst_df)

        unique_id         ds     Prophet
0   AirPassengers 1961-01-01  466.560401
1   AirPassengers 1961-02-01  461.042082
2   AirPassengers 1961-03-01  493.413542
3   AirPassengers 1961-04-01  492.113653
4   AirPassengers 1961-05-01  496.445709
5   AirPassengers 1961-06-01  537.592041
6   AirPassengers 1961-07-01  577.166093
7   AirPassengers 1961-08-01  577.599117
8   AirPassengers 1961-09-01  529.038266
9   AirPassengers 1961-10-01  493.889181
10  AirPassengers 1961-11-01  460.030234
11  AirPassengers 1961-12-01  489.392785
12  AirPassengers 1962-01-01  502.415939
13  AirPassengers 1962-02-01  496.321423
14  AirPassengers 1962-03-01  531.969966
15  AirPassengers 1962-04-01  528.065107
16  AirPassengers 1962-05-01  534.174659
17  AirPassengers 1962-06-01  573.615281
18  AirPassengers 1962-07-01  614.245102
19  AirPassengers 1962-08-01  614.206790
20  AirPassengers 1962-09-01  566.306418
21  AirPassengers 1962-10-01  530.606803
22  AirPassengers 1962-11-01  497.766797
23  AirPassenger

## Ask Questions About Your Forecasts

One of TimeCopilot's powerful features is the ability to answer natural language questions about your forecasts. The `query` method uses the LLM to understand your question and provide insights based on the forecast data.

This demonstrates how the LLM provider you chose is actively being used to understand and respond to your queries.

In [ ]:
query_result = tc.query("how many passengers will be in total in the next months?")
print(query_result.output)

To calculate the total number of passengers forecasted for the next months using the Prophet model, I will sum up the forecasted values. Here are the forecasted passenger numbers for the next several months:

1. 466.56
2. 461.04
3. 493.41
4. 492.11
5. 496.45
6. 537.59
7. 577.17
8. 577.60
9. 529.04
10. 493.89
11. 460.03
12. 489.39

Adding these values gives us a total of approximately \( 6,074.98 \) passengers forecasted over the next 12 months.

This forecast is based on the Prophet model, which has a MASE score of 1.09, indicating its estimation accuracy is relatively good compared to other models.


# Model Compatibility Requirements

## Tool Use is Required

TimeCopilot requires models that support **tool use** (also known as function calling). This is a critical feature that allows the LLM to:
- Call forecasting functions and methods
- Execute code to analyze time series
- Interact with statistical models
- Generate structured outputs

### What This Means

Not every LLM model supports tool use. When choosing a model provider, ensure that:
1. The model supports function calling/tool use
2. The provider's API exposes this capability
3. Pydantic AI can access this feature through the provider

### Compatible Models

Most modern, capable models support tool use, including:
- **OpenAI models**: GPT-4, GPT-4o, GPT-3.5-turbo (with function calling)
- **Ollama models**: Many models support tool use (check model documentation)
- **OpenAI-compatible endpoints**: If the endpoint supports function calling

### Testing Compatibility

If you're unsure whether a model supports tool use, try initializing TimeCopilot with it. If the model doesn't support tool use, you'll receive an error when attempting to use TimeCopilot's features.

### Getting Help

If you encounter compatibility issues:
1. Check the model provider's documentation for tool use/function calling support
2. Verify that Pydantic AI supports your provider
3. Consider using a known-compatible model like GPT-4o or GPT-4

---

## Summary

This notebook has demonstrated how TimeCopilot can work with different LLM providers, giving you flexibility in choosing the right model for your needs. Whether you prefer cloud-based solutions, self-hosted models, or custom endpoints, TimeCopilot provides a consistent interface that works across providers.

**Key Takeaways**:
- TimeCopilot supports any LLM provider compatible with Pydantic AI
- Configuration is done through environment variables
- The interface remains consistent regardless of provider
- Models must support tool use/function calling
- You can easily switch between providers by changing the `llm` parameter

For more information, visit:
- [Pydantic AI Documentation](https://ai.pydantic.dev/)
- [TimeCopilot Documentation](https://timecopilot.dev/)